In [ ]:
import pandas as pd
from tqdm import tqdm

def defence_process_data(df):
    # Initialize columns with 0
    df['Buyer_Profit'] = 0
    df['Buyer_Loss'] = 0
    df['Seller_Profit'] = 0
    df['Seller_Loss'] = 0

    # Convert 'Time Stamp' to datetime for efficient comparison
    df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

    # Use a list to collect results for better performance
    results = []

    # Iterate through each row to perform the logic
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing Rows'):
        # Calculate Buyer Profit and Loss
        if row['BuyersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['defence_target']

            if target_value > entry_value:
                buyer_profit = target_value - entry_value
                buyer_loss = 0
            else:
                buyer_profit = 0
                buyer_loss = abs(target_value - entry_value)
        else:
            buyer_profit = 0
            buyer_loss = 0

        # Calculate Seller Profit and Loss
        if row['SellersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['defence_target']

            if entry_value > target_value:
                seller_profit = entry_value - target_value
                seller_loss = 0
            else:
                seller_profit = 0
                seller_loss = abs(entry_value - target_value)
        else:
            seller_profit = 0
            seller_loss = 0

        # Append results
        results.append({
            'index': index,
            'Buyer_Profit': buyer_profit,
            'Buyer_Loss': buyer_loss,
            'Seller_Profit': seller_profit,
            'Seller_Loss': seller_loss
        })

    # Update DataFrame with results
    results_df = pd.DataFrame(results)
    df.update(results_df.set_index('index'))
    df['Profit'] = df['Buyer_Profit'] + df['Seller_Profit']
    df['Loss'] = df['Buyer_Loss'] + df['Seller_Loss']
    df.drop(columns=["Buyer_Profit","Buyer_Loss","Seller_Profit","Seller_Loss"],axis=1,inplace=True)


    return df